In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
import sastvd as svd
import sastvd.helpers.git as svdg
#svd.get_dir


from multiprocessing import Pool
from tqdm import tqdm

In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from IPython.display import display

In [4]:
#Define a function

def remove_comments(text):
    """Delete comments from code."""

    def replacer(match):
        s = match.group(0)
        if s.startswith("/"):
            return " "  # note: a space and not an empty string
        else:
            return s

    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE,
    )
    return re.sub(pattern, replacer, text)

In [5]:
# This function is copied from sastvd


def dfmp(df, function, columns=None, ordr=True, workers=6, cs=10, desc="Run: "):
    """Parallel apply function on dataframe.

    Example:
    def asdf(x):
        return x

    dfmp(list(range(10)), asdf, ordr=False, workers=6, cs=1)
    """
    if isinstance(columns, str):
        items = df[columns].tolist()
    elif isinstance(columns, list):
        items = df[columns].to_dict("records")
    elif isinstance(df, pd.DataFrame):
        items = df.to_dict("records")
    elif isinstance(df, list):
        items = df
    else:
        raise ValueError("First argument of dfmp should be pd.DataFrame or list.")

    print("huifhuifohuofhweofh hfgfuhfuifgh hfuhyfuhyf")
    
    processed = []
    desc = f"({workers} Workers) {desc}"
    with Pool(processes=workers) as p:
        map_func = getattr(p, "imap" if ordr else "imap_unordered")
        for ret in tqdm(map_func(function, items, cs), total=len(items), desc=desc):
            processed.append(ret)
            
        
    return processed


In [6]:
def bigvul(minimal=True, sample=False, return_raw=False, splits="default"):
    savedir = svd.get_dir(svd.cache_dir() / "minimal_datasets")

    if minimal:
        try:
            df = pd.read_parquet(savedir/ f"minimal_bigvul_{sample}.pq", engine="fastparquet").dropna()
            md = pd.read_csv(svd.cache_dir() / "bigvul/bigvul_metadata.csv")
            md.groupby("project").count().sort_values("id")

            default_splits = external_dir() / "bigvul_rand_splits.csv"

            if os.path.exists(default_splits):
                #I renamed splits as splits_df for better understanding
                splits_df = pd.read_csv(default_splits)
                # splits_df = splits_df.head(10) ##Liza taking only the first 10 rows
                splits_dic    = splits_df.set_index("id").to_dict()["label"]
                #print(splits_dic)
                #print(df.head(15))
                #Liza with this statement they are replacing the old id, label columns with the given id, column from default_splits. 
                #Replacing df[label] with the value 'id' is mapping to in splits_dic
                df["label"] = df.id.map(splits_dic)
                #print(df.head(35))

            if "crossproject" in splits_dic:
                print("crossproject")

            return df

        except Exception as E:
            print(E)
            pass
    
    filename = "MSR_data_cleaned_SAMPLE.csv" if sample else "MSR_data_cleaned.csv"
    print(filename)
    df = pd.read_csv(svd.external_dir() / filename)
    #Renaming the unnamed column to id
    df = df.rename(columns={"Unnamed: 0": "id"})
    df["dataset"] = "bigvul"
    
    
    #display(df.head(10))
    list(df.columns)
    df.describe(include='all').T
    
    
    #Remove Comments
    df["func_before"] = dfmp(df, remove_comments, "func_before", cs=500)
    df["func_after"] = dfmp(df, remove_comments, "func_after", cs=500)
    
    # Return raw for testing
    if return_raw:
        return df
    
    # save codediffs
    cols = ["func_before", "func_after", "id", "dataset"]
    dfmp(df, svdg._c2dhelper, columns=cols, ordr=False, cs = 300)
    
    # Assign info and save
    df["info"] = dfmp(df, svdg.allfunc, cs=500)
    df = pd.concat([df, pd.json_normalize(df["info"])], axis=1)

    return df


In [7]:
df = bigvul(minimal=False)

MSR_data_cleaned.csv


/tmp/ipykernel_2206063/3022217690.py:35: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(svd.external_dir() / filename)


huifhuifohuofhweofh hfgfuhfuifgh hfuhyfuhyf


(6 Workers) Run: 100%|██████████| 188636/188636 [00:00<00:00, 289925.57it/s]


huifhuifohuofhweofh hfgfuhfuifgh hfuhyfuhyf


(6 Workers) Run: 100%|██████████| 188636/188636 [00:00<00:00, 303565.15it/s]


huifhuifohuofhweofh hfgfuhfuifgh hfuhyfuhyf


(6 Workers) Run: 100%|██████████| 188636/188636 [00:01<00:00, 95783.56it/s] 


huifhuifohuofhweofh hfgfuhfuifgh hfuhyfuhyf


(6 Workers) Run: 100%|██████████| 188636/188636 [00:02<00:00, 64735.30it/s]


In [8]:
df.head(10)

,id,Access Gained,Attack Origin,Authentication Required,Availability,CVE ID,CVE Page,CWE ID,Complexity,Confidentiality,...,project_before,vul,vul_func_with_fix,dataset,info,diff,added,removed,before,after
0,0,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static bool check_rodc_critical_attribute(stru...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static bool check_rodc_critical_attribute(stru...,static bool check_rodc_critical_attribute(stru...
1,1,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_add_entry(struct samldb_ctx ...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_add_entry(struct samldb_ctx ...,static int samldb_add_entry(struct samldb_ctx ...
2,2,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_add_entry_callback(struct ld...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_add_entry_callback(struct ld...,static int samldb_add_entry_callback(struct ld...
3,3,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_add_handle_msDS_IntId(struct...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_add_handle_msDS_IntId(struct...,static int samldb_add_handle_msDS_IntId(struct...
4,4,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_add_step(struct samldb_ctx *...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_add_step(struct samldb_ctx *...,static int samldb_add_step(struct samldb_ctx *...
5,5,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_allocate_sid(struct samldb_c...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_allocate_sid(struct samldb_c...,static int samldb_allocate_sid(struct samldb_c...
6,6,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_check_sAMAccountName(struct ...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_check_sAMAccountName(struct ...,static int samldb_check_sAMAccountName(struct ...
7,7,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_check_user_account_control_i...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static int samldb_check_user_account_control_i...,static int samldb_check_user_account_control_i...
8,8,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static struct samldb_ctx *samldb_ctx_init(stru...,bigvul,"{'diff': '', 'added': [], 'removed': [], 'befo...",,[],[],static struct samldb_ctx *samldb_ctx_init(stru...,static struct samldb_ctx *samldb_ctx_init(stru...
9,9,None,Remote,Single system,Partial,CVE-2015-8467,https://www.cvedetails.com/cve/CVE-2015-8467/,CWE-264,Medium,Partial,...,https://git.samba.org/?p=samba.git;a=blob;f=so...,0,static int samldb_fill_foreignSecurityPrincipa...,bigvul,"{'diff': '', 'added': [

In [ ]:
df.describe(include='all').T

In [ ]:
df.isna().sum()

In [ ]:
svd.hello()